In [ ]:
from csv import reader
from collections import defaultdict
from itertools import chain, combinations

In [ ]:
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

In [ ]:
def getFromFile(fname):
    itemSets = []
    itemSet = set()
    with open(fname, "r") as file:
        csv_reader = reader(file)
        for line in csv_reader:
            line = list(filter(None, line))
            record = set(line)
            for item in record:
                itemSet.add(frozenset([item]))
            itemSets.append(record)
    return itemSet, itemSets

In [ ]:
def getAboveMinSup(itemSet, itemSetList, minSup, globalItemSetWithSup):
    freqItemSet = set()
    localItemSetWithSup = defaultdict(int)
    for item in itemSet:
        for itemSet in itemSetList:
            if item.issubset(itemSet):
                globalItemSetWithSup[item] += 1
                localItemSetWithSup[item] += 1
    for item, supCount in localItemSetWithSup.items():
        support = float(supCount / len(itemSetList))
        if support >= minSup:
            freqItemSet.add(item)
    return freqItemSet

In [ ]:
def getUnion(itemSet, length):
    return set(
        [i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length]
    )

In [ ]:
def pruning(candidateSet, prevFreqSet, length):
    tempCandidateSet = candidateSet.copy()
    for item in candidateSet:
        subsets = combinations(item, length)
        for subset in subsets:
            if frozenset(subset) not in prevFreqSet:
                tempCandidateSet.remove(item)
                break
    return tempCandidateSet

In [ ]:
def associationRule(freqItemSet, itemSetWithSup, minConf):
    rules = []
    for k, itemSet in freqItemSet.items():
        for item in itemSet:
            subsets = powerset(item)
            for s in subsets:
                confidence = float(itemSetWithSup[item] / itemSetWithSup[frozenset(s)])
                if confidence > minConf:
                    rules.append([set(s), set(item.difference(s)), confidence])
    return rules

In [ ]:
def getItemSetFromList(itemSetList):
    tempItemSet = set()
    for itemSet in itemSetList:
        for item in itemSet:
            tempItemSet.add(frozenset([item]))
    return tempItemSet

In [ ]:
def apriori(fname, minSup, minConf):
    C1ItemSet, itemSetList = getFromFile(fname)
    globalFreqItemSet = dict()
    globalItemSetWithSup = defaultdict(int)
    L1ItemSet = getAboveMinSup(C1ItemSet, itemSetList, minSup, globalItemSetWithSup)
    currentLSet = L1ItemSet
    k = 2
    while currentLSet:
        globalFreqItemSet[k - 1] = currentLSet
        candidateSet = getUnion(currentLSet, k)
        candidateSet = pruning(candidateSet, currentLSet, k - 1)
        currentLSet = getAboveMinSup(
            candidateSet, itemSetList, minSup, globalItemSetWithSup
        )
        k += 1
    rules = associationRule(globalFreqItemSet, globalItemSetWithSup, minConf)
    rules.sort(key=lambda x: x[2])
    return globalFreqItemSet, rules

In [ ]:
if __name__ == "__main__":
    freqItemSet, rules = apriori("DataSet1.csv", 0.2, 0.5)
    for i in freqItemSet:
        print(str(i) + " : ")
        for j in freqItemSet[i]:
            print(j)
        print("--------------------------------------------------------")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    for i in rules:
        print(i)